In [ ]:
# we want a read and write for the bulk data
# there will be different read and write functions for different data types

# in the end data will be read in based on the schema of the export file
    # but then put into the bulk data schema and with the normal schema

# writing 5 minute data files to the right location will be a part of it
# splitting the input data into the 5 minute segments will also be a part of it 


In [1]:
# so for the recent captures what would I do
# I'll have a dictionary of camera names and metadata for the bulk data location

# I'll go through a sorted list of the files
# I'll grab the device name from the folder name
# I'll go into the folder and combine all the parquets and write < ez jk I need these for the quereies
# for the bulk data, read it one file at a time and query the data for each 5 minute segment of the day
# this can use the same query function that could be used for querying the bulk data

# it will take in a bounds of times and retrun the frames between those times
# then a different function will take those frames and write them

# it does involve some way of linking the data to the time associated with the data
# let's just start with the video data I have 

# for every 5 minute segment
# search the deduplicated file names for the relevant files
# read the parquet and line up the times with the frame counts

# the metadata should have the time, but ngl it could only have the time
# we could get the frame offset by querieng the times for the 5 minute segment
# and then lining up the indexes

# in that case we can just write all the files and hope they line up
# although taking it file by file might help keep errors from propigating
# if I had good tests I could trust this would be less of an issue
# acctually if the timestamps line up there's no issues
# this is only a problem for the old capture data beofre I fixed that timestamp bug
# I will still write it file by file to make it so I only have to write one 

# this schema does mean that the files themselves will be of gratly varying length due to the timestamps
# as little as 20 frames per video in timelapse mode which would result in low compression
# this is why we had the heterogenious timesamp structure anyway

# the essence then could be the same thing, but we treat the file names as the indcators for where the data is
# then we query the timestampsdf for that videos frames and calculate offsets for the desired times
# return the frames that are 

# so the takeaway from this is that we can just cut off by day (which will still take some code for audio)
# but for the camera structure we don't need to do anything since it's already divided that way.

# so back to the standard way I was doing it where I just transfer all the videos as they are to the new day folder
# I can also just add all the timestamps to the metadata raw and just make a check incase there are too many timestamps



In [1]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import pandas as pd
import os
import sys
import subprocess
import cv2
import shutil

def getRepoPath():
    cwd = os.getcwd()
    delimiter = "\\" if "\\" in cwd else "/"
    repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
    return repoPath
repoPath = getRepoPath()
sys.path.append(repoPath + "dataImport/")
import rwWorkingTSDf
from rwWorkingTSDf import writeWorkingTSDf, readWorkingTSDF, fnString_to_dt, dt_to_fnString
recentCapPath = repoPath + "recentCaptures/"

In [2]:
# dictionaries
from collections import OrderedDict
fieldNames = ["responsiblePartyName", "instanceName", "developingPartyName", "deviceName", "dataType", "dataSource"]
myCams = {}
myCams["mobilepi"] = ["abhik", "mobilepi", "abhik", "piCam-raspberryPi5-Camv3120noir", "mp4", "piVidCap"]
myCams["bedroompi"] = ["abhik", "bedroompi", "abhik", "piCam-raspberryPi5-Camv3120", "mp4", "piVidCap"]
myCams["bathroompi"] = ["abhik", "bathroompi", "abhik", "piCam-raspberryPi5-Camv3noir", "mp4", "piVidCap"]
myCams["bathroomCam"] = ["abhik", "bathroomCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]
myCams["deskCam"] = ["abhik", "deskCam", "abhik", "pcCam-webcam", "mp4", "vidCap"]
# right now we are going through and writing the intake code for recent video captures
# later I would like to refactor the pividcap and the normal vidcap
# and clear out everything un needed for video capture in the repo
# I honestly should be writing this in data import


# damn I have a bunch of files seperated by the current timezone day and not the UTC day
# this does mean that there will be some cutting together and apart of the videos

In [3]:
def geBulkLocation(time, responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource):
    dataFolderName = "_".join([responsiblePartyName, instanceName, developingPartyName, deviceName, dataType, dataSource]) + "/"
    dataFolderName += time.strftime("%Y") + "/" + time.strftime("%m-%d") + "/"
    return repoPath + "bulkData/" + dataFolderName

In [4]:
def frame_to_time(frame_index, fps):
    """Convert frame index to timestamp format HH:MM:SS.milliseconds"""
    seconds = frame_index / fps
    return f"{int(seconds // 3600):02}:{int((seconds % 3600) // 60):02}:{seconds % 60:.3f}"

def splitVid(input_file_location, boundary_indexes, output_paths):
    print(input_file_location)
    cap = cv2.VideoCapture(input_file_location)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    cap.release()

    for i, (start_frame, end_frame) in enumerate(boundary_indexes):
        start_time = frame_to_time(start_frame, fps)
        end_time = frame_to_time(end_frame, fps)
        
        command = [
            "ffmpeg", "-y", "-i", input_file_location,
            "-ss", start_time, "-to", end_time,
            "-c", "copy", output_paths[i]
        ]
        print(command)
        subprocess.run(command)

def fnString_to_dt(fn_string):
    fn_string = fn_string[:17] + ',' + fn_string[18:]
    return datetime.fromisoformat(fn_string.replace(",", "."))



In [5]:

# read in the metadata parquet
workingdfs = {}
for k in myCams.keys():
    workingdfs[k] = readWorkingTSDF(*myCams[k])


foldersToImport = sorted(os.listdir(recentCapPath))
for f in foldersToImport:
    camName = f.split("_")[0] ################################################ change all of these once we update the vid cap
    folderPath = recentCapPath + f + "/"
    # get a list of file name bases (withouth the file types)
    fileNameBases = sorted(list(set([x.split(".")[0] for x in os.listdir(folderPath)])))
    
    # combine all parquets
    for i, fileName in enumerate(fileNameBases):
        source = folderPath + fileName + ".parquet"
        if i == 0:
            readDf = pd.read_parquet(source)
        else:
            readDf = pd.concat([readDf, pd.read_parquet(source)])

    # add to the full metadata parquet
    if type(workingdfs[camName]) == type(None):
        workingdfs[camName] = readDf
    else:
        workingdfs[camName] = pd.concat([workingdfs[camName], readDf]).sort_index()

    # for each file in a folder
    for fileName in fileNameBases:
        startTime = fnString_to_dt(fileName.split("_")[-2]).astimezone(ZoneInfo("UTC"))
        endTime = fnString_to_dt(fileName.split("_")[-1]).astimezone(ZoneInfo("UTC"))
        # if not a boundary file rename, move and contuinue
        #print(startTime.day)
        #print(endTime.day)
        if startTime.day == endTime.day:
            # generate new file name
            newFileName = dt_to_fnString(startTime) + "_" + dt_to_fnString(endTime) + ".mp4"
            source = folderPath + fileName + ".mp4"
            destination = geBulkLocation(startTime, *myCams[camName])
            # move to the right location
            #print(destination)
            #print(os.path.exists(destination))
            if not os.path.exists(destination):
                print("made directory " + destination)
                os.makedirs(destination)
            shutil.copy2(source, destination  + newFileName)
            continue

        # the file contains more than one day of data
        frameTimes = workingdfs[camName][startTime:endTime].copy()
        frameTimes['date'] = frameTimes.index.normalize()
        boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]))
        index_boundaries = [(frameTimes.index.get_loc(start), frameTimes.index.get_loc(end))
                                for day, (start, end) in boundaries.items()]
        
        output_paths = [geBulkLocation(x[0], *myCams[camName]) + 
                        dt_to_fnString(x[0]) + "_" + dt_to_fnString(x[1]) + ".mp4" 
                        for x in boundaries]

        splitVid(folderPath + fileName  + ".mp4", index_boundaries, output_paths)

    # delete the folder
    #os.rmdir(recentCapPath + f)




no folder with the location /home/chowder/Documents/workingData/abhik_mobilepi_abhik_piCam-raspberryPi5-Camv3120noir_mp4_piVidCap/
no folder with the location /home/chowder/Documents/workingData/abhik_bedroompi_abhik_piCam-raspberryPi5-Camv3120_mp4_piVidCap/
no folder with the location /home/chowder/Documents/workingData/abhik_bathroompi_abhik_piCam-raspberryPi5-Camv3noir_mp4_piVidCap/
no folder with the location /home/chowder/Documents/workingData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/
no folder with the location /home/chowder/Documents/workingData/abhik_deskCam_abhik_pcCam-webcam_mp4_vidCap/
made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/12-11/
/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-11-0700/bathroomCam_2024-12-11T112945-000174-0700_2024-12-11T170102-700091-0700.mp4
30.0
['ffmpeg', '-y', '-i', '/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-11-0700/bathroomCam_2024-12-11T112945-000174-0700_

/tmp/ipykernel_3792680/2936753165.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]))
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock

made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/12-13/
/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-13-0700/bathroomCam_2024-12-13T165952-300136-0700_2024-12-13T170252-200135-0700.mp4
30.0
['ffmpeg', '-y', '-i', '/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-13-0700/bathroomCam_2024-12-13T165952-300136-0700_2024-12-13T170252-200135-0700.mp4', '-ss', '00:00:0.000', '-to', '00:00:59.967', '-c', 'copy', '/home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/12-13/2024-12-13T235952,300136+0000_2024-12-14T000252,200135+0000.mp4']
made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/12-14/


/tmp/ipykernel_3792680/2936753165.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]))
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock

/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-14-0700/bathroomCam_2024-12-14T165230-000177-0700_2024-12-14T225704-200143-0700.mp4
30.0
['ffmpeg', '-y', '-i', '/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-14-0700/bathroomCam_2024-12-14T165230-000177-0700_2024-12-14T225704-200143-0700.mp4', '-ss', '00:00:0.000', '-to', '00:00:59.967', '-c', 'copy', '/home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/12-14/2024-12-14T235230,000177+0000_2024-12-15T055704,200143+0000.mp4']
made directory /home/chowder/Documents/bulkData/abhik_bathroomCam_abhik_pcCam-webcam_mp4_vidCap/2024/12-15/
/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-15-0700/bathroomCam_2024-12-15T142628-200093-0700_2024-12-15T173159-600121-0700.mp4
30.0
['ffmpeg', '-y', '-i', '/home/chowder/Documents/recentCaptures/bathroomCam_2024-12-15-0700/bathroomCam_2024-12-15T142628-200093-0700_2024-12-15T173159-600121-0700.mp4', '-ss', '00:00:0.000', '-to', '00:00:59.9

...

/tmp/ipykernel_3792680/2936753165.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]))
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock

/home/chowder/Documents/recentCaptures/mobilepi_2025-02-07-0700/mobilepi_2025-02-07T164748-601157-0700_2025-02-07T184722-400068-0700.mp4
30.0
['ffmpeg', '-y', '-i', '/home/chowder/Documents/recentCaptures/mobilepi_2025-02-07-0700/mobilepi_2025-02-07T164748-601157-0700_2025-02-07T184722-400068-0700.mp4', '-ss', '00:00:0.000', '-to', '00:00:59.967', '-c', 'copy', '/home/chowder/Documents/bulkData/abhik_mobilepi_abhik_piCam-raspberryPi5-Camv3120noir_mp4_piVidCap/2025/02-07/2025-02-07T234748,601157+0000_2025-02-08T014722,400068+0000.mp4']
made directory /home/chowder/Documents/bulkData/abhik_mobilepi_abhik_piCam-raspberryPi5-Camv3120noir_mp4_piVidCap/2025/02-08/


/tmp/ipykernel_3792680/2936753165.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]))
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock

/home/chowder/Documents/recentCaptures/mobilepi_2025-02-08-0700/mobilepi_2025-02-08T155340-700065-0700_2025-02-08T171231-800111-0700.mp4
30.0
['ffmpeg', '-y', '-i', '/home/chowder/Documents/recentCaptures/mobilepi_2025-02-08-0700/mobilepi_2025-02-08T155340-700065-0700_2025-02-08T171231-800111-0700.mp4', '-ss', '00:00:0.000', '-to', '00:00:59.967', '-c', 'copy', '/home/chowder/Documents/bulkData/abhik_mobilepi_abhik_piCam-raspberryPi5-Camv3120noir_mp4_piVidCap/2025/02-08/2025-02-08T225340,700065+0000_2025-02-09T001231,800111+0000.mp4']
made directory /home/chowder/Documents/bulkData/abhik_mobilepi_abhik_piCam-raspberryPi5-Camv3120noir_mp4_piVidCap/2025/02-09/
/home/chowder/Documents/recentCaptures/mobilepi_2025-02-09-0700/mobilepi_2025-02-09T163016-600148-0700_2025-02-09T172100-700163-0700.mp4
30.0
['ffmpeg', '-y', '-i', '/home/chowder/Documents/recentCaptures/mobilepi_2025-02-09-0700/mobilepi_2025-02-09T163016-600148-0700_2025-02-09T172100-700163-0700.mp4', '-ss', '00:00:0.000', '-to',

/tmp/ipykernel_3792680/2936753165.py:52: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  boundaries = frameTimes.groupby('date').apply(lambda x: (x.index[0], x.index[-1]))
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1729892860555/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock